In [1]:
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import math
from matplotlib import pyplot as plt

## Load Dataset (Iris)

In [2]:
iris_df = pd.read_csv("iris.csv", names=["sepal_length", "sepal_width", "petal_length", "petal_width", "class"])
with pd.option_context('future.no_silent_downcasting', True):
    iris_df= iris_df.replace({'Iris-setosa':0, 'Iris-versicolor': 1, 'Iris-virginica': 2}).infer_objects()

In [3]:
iris_df

,sepal_length,sepal_width,petal_length,petal_width,class
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


## Dataset Processing

In [4]:
feature_cols = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
target_col = 'class'

X = np.array(iris_df[feature_cols].values)
y = np.array(iris_df[target_col].values)

In [5]:
#Using Kfolds
kfolds = KFold(n_splits=10, random_state=None, shuffle=True)

## Entropy, Information Gain, Node Class, and DecisionTree Class Implementation

In [6]:
def entropy(y):
    count = np.bincount(y) #Counting how many of each class occurs
    probs = count / len(y) #Calculating the probabilities of each respective class
    return -sum(p * np.log2(p) for p in probs if p > 0) #Entropy calculation. When log2(0), it is infinity, but p is 0, resulting in 0. hence, with the if statement, skip to avoid errors.
    
def information_gain(X_column, y, threshold):
    parent_entropy = entropy(y) #Calculating parent entropy
    l_mask, r_mask = X_column <= threshold, X_column > threshold #Creating a boolean array to classify elements to left or right subtree. 
    l_child, r_child = y[l_mask], y[r_mask] #Classifying left and right child using boolean array retrieved from previous line
    l_entropy, r_entropy = entropy(l_child), entropy(r_child) #Calculating the respective entropies of left and right child
    weighted_child_entropy = (len(l_child)/len(y)) * l_entropy + (len(r_child)/len(y)) * r_entropy #Calculating weighted entropy. Weight is the proportion of the subtree against all.

    return parent_entropy - weighted_child_entropy #Calculating then returning information gain.

class Node: #Node class for decision tree implementation
    def __init__(self, feature_index = None, threshold = None, left = None, right = None, value = None ):
        self.feature_index = feature_index #To denote which feature was used for threshold
        self.threshold = threshold #Threshold of the split
        self.left = left #Left subtree
        self.right = right #Right subtree
        self.value = value #Value. When value = None, it is one of the inner nodes. If value equals to some class, it is a leaf node.
        
class DecisionTree: #Decision tree class
    def __init__(self, root=None, nmin = None):
        self.root = root #Setting root of decision tree
        self.nmin = nmin #Declaring Nmin for the decision tree

    def best_threshold(self, X, y, num_features): #Calculating the best threshold
        best_gain = -float('inf')
        best_threshold = None
        best_feature = None
        
        for feature_index in range(num_features): #Itearting through all possible features
            thresholds = np.unique(X[:,feature_index]) #Sorting and removing duplicate values. Sorting is necessary for midpoint calculation.
            thresholds = (thresholds[1:] + thresholds[:-1])/2 #Calculating all possible midpoints.
            
            for threshold in thresholds:#Iterating through all possible thresholds with respect to the current feature value.
                gain = information_gain(X[:,feature_index], y, threshold) #Calculating the information gain with respect to the feautre and its threshold.

                if gain > best_gain: #If current gain is greater than the overall information gain, meaning its the best threshold.
                    best_gain = gain #Updating best information gain
                    best_threshold = threshold #Updating best threshold
                    best_feature = feature_index #Updating which feature the threshold is applied to.

        return best_feature, best_threshold #Return the best feature with the best threshold.

    def fit_tree(self, X, y): #Training the decision tree.
        if len(y) < self.nmin or len(set(y)) == 1: #Base case: split n < nmin or when there are only one type of class
            values, counts = np.unique(y, return_counts = True)
            return Node(value = values[np.argmax(counts)]) #Calculating which class is dominant, and setting this leaf node to that class.

        best_feature, best_threshold = self.best_threshold(X,y, len(X[0])) #Calculating the best feature and threshold from all possible feature and thresholds.

        if best_threshold is None or best_feature is None: #If there are no further splits available, it is a leaf node.
            values, counts = np.unique(y, return_counts = True)
            return Node(value = values[np.argmax(counts)]) #Calculating which class is dominant, and setting this leaf node to that class.

        left_mask = X[:, best_feature] <= best_threshold #Dividing data to left and right child, based on the boolean array calculated.
        right_mask = X[:, best_feature] > best_threshold

        l_child, r_child = y[left_mask], y[right_mask]

        if len(l_child) < self.nmin or len(r_child) < self.nmin: #If the split creates a child with n < self.nmin, stop splitting
            values, counts = np.unique(y, return_counts = True)
            return Node(value = values[np.argmax(counts)]) #Calculating which class is dominant, and setting this leaf node to that class.
        
        if len(l_child) == 0 or len(r_child) == 0: #If either left or right child is empty, it is a leaf node.
            values, counts = np.unique(y, return_counts = True)
            return Node(value = values[np.argmax(counts)]) #Calculating which class is dominant, and setting this leaf node to that class.
            
        l_subtree = self.fit_tree(X[left_mask], y[left_mask]) #Recursively creating left and right subtree.
        r_subtree = self.fit_tree(X[right_mask], y[right_mask])

        return Node(feature_index = best_feature, threshold = best_threshold, left = l_subtree, right = r_subtree) #Returning the root of the decision tree.

    def predict(self, x, node): #Predicting sample by sample by iterating through the decision tree and finding which leaf node it belongs to. Return the class of the leaf node.
        if node.value is not None: #Base case: leaf node reached. Return the node value.
            return node.value
        if x[node.feature_index] <= node.threshold: #Move to left subtree
            return self.predict(x, node.left)
        else:
            return self.predict(x, node.right) #Move to right subtree
        

## Training and Testing Iris Dataset

In [7]:
nmins = [5, 10, 15, 20]
accuracy_scores_per_nmin = []

for nmin in nmins: #Training and testing Iris dataset on different nmins.
    accuracy_scores = [] #Scoring the acccuracies across all kfolds.

    for train_idx, test_idx in kfolds.split(X):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
    
        iris_model = DecisionTree(nmin = nmin) #Creating a decision tree of nmin = nmin.
        iris_model.root = iris_model.fit_tree(X_train, y_train) #Fitting tree.
        y_pred = []
        for x in X_test: #Predicting element by element
            y_pred.append(iris_model.predict(x, iris_model.root)) #Storing the prediction on the y_pred array.
        accuracy = accuracy_score(y_test, y_pred) #Calculating accuracy.
        accuracy_scores.append(accuracy) #Storing accuracy of each fold in accuracy_scores
        
    print(f"Accuracy scores for nmin = {nmin}: {accuracy_scores}")
    accuracy_scores_per_nmin.append(accuracy_scores) #Storing the accuracy of the nmin across all folds.

Accuracy scores for nmin = 5: [1.0, 1.0, 1.0, 0.8, 1.0, 0.9333333333333333, 0.8666666666666667, 0.8666666666666667, 1.0, 0.8666666666666667]
Accuracy scores for nmin = 10: [1.0, 0.8, 1.0, 1.0, 0.8666666666666667, 0.9333333333333333, 1.0, 0.9333333333333333, 1.0, 0.9333333333333333]
Accuracy scores for nmin = 15: [1.0, 1.0, 0.9333333333333333, 0.8666666666666667, 1.0, 1.0, 0.8666666666666667, 0.9333333333333333, 0.9333333333333333, 0.9333333333333333]
Accuracy scores for nmin = 20: [0.9333333333333333, 0.9333333333333333, 1.0, 1.0, 0.8666666666666667, 0.9333333333333333, 1.0, 0.9333333333333333, 1.0, 0.9333333333333333]


## Results of Iris Dataset Decision Tree

In [8]:
iris_average_accuracy = [np.mean(acc) for acc in accuracy_scores_per_nmin] #Calculating average accuracy for each nmin across the folds
iris_variance = [np.var(acc) for acc in accuracy_scores_per_nmin] #Calculating variance for each nmin across the folds

iris_model_data = { #Creating a pandas dataframe to better visualize as a table
    "Nmin" : nmins,
    "Accuracy" : iris_average_accuracy,
    "Variance" : iris_variance
}

iris_model_df = pd.DataFrame(iris_model_data)

print(iris_model_df)

   Nmin  Accuracy  Variance
0     5  0.933333  0.005333
1    10  0.946667  0.004267
2    15  0.946667  0.002489
3    20  0.953333  0.001822


Nmin's effect can be clearly seen in models regarding the iris dataset. Since the sample size is small (150), the affect of nmin is large. When nmin is small, the decision tree will go deep, overfitting with high variance. When nim is large, the tree will be shallow, underfitting with low variance. This trend can clearly be seen as nmin grows from 5 to 20, where nimin = 5 has the lowerst accuracy and highest variance, while nmin = 20 has the highest accuracy with the lowest variance.

## Load Dataset (Spambase)

In [9]:
spambase_df = pd.read_csv("spambase.csv")

In [10]:
spambase_df

,0,0.64,0.64.1,0.1,0.32,0.2,0.3,0.4,0.5,0.6,...,0.41,0.42,0.43,0.778,0.44,0.45,3.756,61,278,1
0,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.000,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
1,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.010,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
2,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.000,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.000,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,1.85,0.00,0.00,1.85,0.00,0.00,...,0.000,0.223,0.0,0.000,0.000,0.000,3.000,15,54,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4595,0.31,0.00,0.62,0.0,0.00,0.31,0.00,0.00,0.00,0.00,...,0.000,0.232,0.0,0.000,0.000,0.000,1.142,3,88,0
4596,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.000,0.000,0.0,0.353,0.000,0.000,1.555,4,14,0
4597,0.30,0.00,0.30,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.102,0.718,0.0,0.000,0.000,0.000,1.404,6,118,0
4598,0.96,0.00,0.00,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.000,0.057,0.0,0.000,0.000,0.000,1.147,5,78,0


## Data Processing

In [11]:
X_spam = np.array(spambase_df.iloc[:, :-1])
y_spam = np.array(spambase_df.iloc[:, -1])

## Training and Testing Spambase Dataset

In [12]:
spambase_nmins = [5, 10, 15, 20, 25]
spambase_accuracy_scores_per_nmin = []

for nmin in spambase_nmins: #Training and testing Iris dataset on different nmins.
    accuracy_scores = [] #Scoring the acccuracies across all kfolds.

    for train_idx, test_idx in kfolds.split(X_spam):
        X_train, X_test = X_spam[train_idx], X_spam[test_idx]
        y_train, y_test = y_spam[train_idx], y_spam[test_idx]
    
        spambase_model = DecisionTree(nmin = nmin) #Creating a decision tree of nmin = nmin.
        spambase_model.root = spambase_model.fit_tree(X_train, y_train) #Fitting tree.
        y_pred = []
        for x in X_test:
            y_pred.append(iris_model.predict(x, spambase_model.root)) #Storing the prediction on the y_pred array.
        accuracy = accuracy_score(y_test, y_pred) #Calculating accuracy.
        accuracy_scores.append(accuracy) #Storing accuracy of each fold in accuracy_scores
        
    print(f"Accuracy scores for nmin = {nmin}: {accuracy_scores}")
    spambase_accuracy_scores_per_nmin.append(accuracy_scores) #Storing the accuracy of the nmin across all folds.

Accuracy scores for nmin = 5: [0.9195652173913044, 0.9043478260869565, 0.9369565217391305, 0.9282608695652174, 0.9152173913043479, 0.9434782608695652, 0.9282608695652174, 0.9304347826086956, 0.9217391304347826, 0.8913043478260869]
Accuracy scores for nmin = 10: [0.9152173913043479, 0.9152173913043479, 0.908695652173913, 0.9130434782608695, 0.9282608695652174, 0.9478260869565217, 0.9260869565217391, 0.8934782608695652, 0.9043478260869565, 0.9152173913043479]
Accuracy scores for nmin = 15: [0.9239130434782609, 0.9021739130434783, 0.9021739130434783, 0.9108695652173913, 0.9065217391304348, 0.8891304347826087, 0.9282608695652174, 0.9391304347826087, 0.8934782608695652, 0.9282608695652174]
Accuracy scores for nmin = 20: [0.9369565217391305, 0.9108695652173913, 0.8956521739130435, 0.9130434782608695, 0.9282608695652174, 0.8913043478260869, 0.908695652173913, 0.8869565217391304, 0.9, 0.8804347826086957]
Accuracy scores for nmin = 25: [0.9347826086956522, 0.8956521739130435, 0.8934782608695652

## Results of Spambase Dataset Decision Tree

In [13]:
spambase_average_accuracy = [np.mean(acc) for acc in spambase_accuracy_scores_per_nmin] #Calculating average accuracy for each nmin across the folds
spambase_variance = [np.var(acc) for acc in spambase_accuracy_scores_per_nmin] #Calculating variance for each nmin across the folds

spambase_model_data = { #Creating a pandas dataframe to better visualize as a table
    "Nmin" : spambase_nmins,
    "Accuracy" : spambase_average_accuracy,
    "Variance" : spambase_variance
}

spambase_model_df = pd.DataFrame(spambase_model_data)

print(spambase_model_df)

   Nmin  Accuracy  Variance
0     5  0.921957  0.000214
1    10  0.916739  0.000197
2    15  0.912391  0.000250
3    20  0.905217  0.000290
4    25  0.907826  0.000236


Nmin's effect is not clearly seen in the spambase model. Since the dataset is large (4600 samples), the optimum nmin seems to be larger than 25, as we could see the accuracy went slightly up and variance went slightly down from nmin = 20 to nmin = 25. 